In [1]:
import syft as sy

Login to the network with Data Scientist credentials

Note: Make sure to create the user first through the GUEHDS portal

In [ ]:
data_scientist_client = sy.login(port=8081, email="email", password="pass")
data_scientist_client

Explore the available datasets on the network

In [ ]:
datasets = data_scientist_client.datasets.get_all()
datasets

Select a dataset and the respective mock

In [20]:
dataset = datasets[0]
asset = dataset.assets[0]

In [ ]:
mock = asset.mock
mock

Create a function to analyze the dataset

In [ ]:
@sy.syft_function_single_use(data=asset)
def detections(data):
    positve_tests = sum(data["Positive Tests"])

    return positve_tests


@sy.syft_function_single_use(data=asset)
def positivity(data):
    tests = sum(data["Total Tests"])
    positve_tests = sum(data["Positive Tests"])

    return positve_tests / tests

@sy.syft_function_single_use(data=asset)
def doses(data):
    doses = sum(data["Cumulative_Flu_Doses_Distributed"])

    return doses

In [ ]:
pointer = detections(data=asset)
pointer.get()

Submit function to the Domain Server

In [ ]:
new_project = sy.Project(
    name="Covid Report",
    description="A project looking at some covid data",
    members=[data_scientist_client],
)
new_project

In [ ]:
new_project.create_code_request(detections, data_scientist_client)
new_project.create_code_request(positivity, data_scientist_client)

In [ ]:
data_scientist_client.code

In [ ]:
project = new_project.start()
project

Execute the submitted function and explore the result

In [ ]:
asset = data_scientist_client.datasets[0].assets[0]
data_scientist_client.code.detections(data=asset).get()

As you can see, the result is not ready to be analyzed, because it needs to be approved by the Data Owners.